In [ ]:
!nvidia-smi

Wed Feb 10 16:35:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install tensorflow-gpu

In [ ]:
train_dir = '/content/gdrive/MyDrive/kaggle_datasets/messy-vs-clean-room/images/train' 
test_dir = '/content/gdrive/MyDrive/kaggle_datasets/messy-vs-clean-room/images/test'
val_dir = '/content/gdrive/MyDrive/kaggle_datasets/messy-vs-clean-room/images/val'

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(299,299),
                                                    batch_size=16,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(299,299),
                                                        batch_size=16,
                                                        class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
model = models.Sequential([
                           layers.Conv2D(32, (3,3), activation='relu', input_shape=(299, 299, 3)),
                           layers.MaxPooling2D((2,2)),

                           layers.Conv2D(64, (3,3), activation='relu'),
                           layers.MaxPooling2D((2,2)),

                           layers.Conv2D(64, (3,3), activation='relu'),
                           layers.MaxPooling2D((2,2)),

                           layers.Conv2D(128, (3,3), activation='relu'),
                           layers.MaxPooling2D((2,2)),

                           layers.Conv2D(256, (3,3), activation='relu'),
                           layers.MaxPooling2D((2,2)),

                           layers.Conv2D(256, (3,3), activation='relu'),
                           layers.MaxPooling2D((2,2)),

                           layers.Flatten(),

                           layers.Dense(512, activation='relu'),
                           layers.Dense(1, activation='sigmoid')


])

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate=2e-5),
              tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=12,
                             epochs=30,
                             validation_data=validation_generator,
                             validation_steps=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
12/12 [==============================] - 99s 8s/step - loss: 0.6930 - accuracy: 0.4924 - val_loss: 0.6911 - val_accuracy: 0.6000
Epoch 2/30
12/12 [==============================] - 27s 2s/step - loss: 0.6902 - accuracy: 0.6125 - val_loss: 0.6891 - val_accuracy: 0.5000
Epoch 3/30
12/12 [==============================] - 27s 2s/step - loss: 0.6894 - accuracy: 0.5186 - val_loss: 0.6864 - val_accuracy: 0.5500
Epoch 4/30
12/12 [==============================] - 27s 2s/step - loss: 0.6879 - accuracy: 0.5353 - val_loss: 0.6825 - val_accuracy: 0.6000
Epoch 5/30
12/12 [==============================] - 27s 2s/step - loss: 0.6826 - accuracy: 0.6197 - val_loss: 0.6773 - val_accuracy: 0.6000
Epoch 6/30
12/12 [==============================] - 27s 2s/step - loss: 0.6766 - accuracy: 0.6124 - val_loss: 0.6703 - val_accuracy: 0.6000
Epoch 7/30
12/12 [==============================] - 27s 2s/step - loss: 0.6659 - accuracy: 0.7352 - val_loss: 0.6597 - val_accuracy: 0.6500
Epoch 8/30
12/12 [==

In [ ]:
model.save('custom_messy_vs_clean_room.h5')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

testing_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(299,299),
                                                        batch_size=2,
                                                        class_mode='binary')

Found 20 images belonging to 2 classes.


In [ ]:
test_loss, test_acc = model.evaluate_generator(testing_generator, steps=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


In [ ]:
print('test accuracy = ', test_acc)

test accuracy =  0.75
